In [ ]:
import pandas as pd
from ydata_profiling import ProfileReport

data = pd.read_csv("../../data/structured/carbon.csv")
profile = ProfileReport(data, title="Carbon Emissions Data Profiling Report", explorative=True)
profile.to_file('../../data/files/analysis.html')

In [8]:
data.drop(columns=['TID', 'logger'],inplace=True)

In [9]:
data.head()

,timestamp,level,context,message
0,2024-11-15 15:44:30.720,INFO,org.wso2.carbon.core.internal.CarbonCoreActivator,Starting WSO2 Carbon...
1,2024-11-15 15:44:30.739,INFO,org.wso2.carbon.core.internal.CarbonCoreActivator,"Operating System : Linux 5.4.0-198-generic, amd64"
2,2024-11-15 15:44:30.739,INFO,org.wso2.carbon.core.internal.CarbonCoreActivator,Java Home : /opt/java/openjdk
3,2024-11-15 15:44:30.740,INFO,org.wso2.carbon.core.internal.CarbonCoreActivator,Java Version : 11.0.10
4,2024-11-15 15:44:30.740,INFO,org.wso2.carbon.core.internal.CarbonCoreActivator,Java VM : OpenJDK 64-Bit Server VM 11...
